In [1]:
import os
import sys
import gc
import pickle

from tensorflow import keras
import numpy as np
from tqdm import tqdm

from nea.ml.nn import (
    Module,
    ModuleList,
    Conv2D,
    Dense,
    Reshape,
    ReLU,
    Tanh,
    Reshape,
    MinMaxNormalization,
    SGD,
    MSE,
    AlphaLoss,
    Softmax,
    Sigmoid,
    CrossEntropy,
)
from nea.ml.autograd import Tensor, no_grad

2025-02-20 16:05:14.857004: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-20 16:05:14.907191: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-20 16:05:14.961974: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740067515.043499   27339 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740067515.071613   27339 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-20 16:05:15.227856: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [2]:
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

"""train_filter = np.where((y_train == 0) | (y_train == 4))
test_filter = np.where((y_test == 0) | (y_test == 4))

X_train, y_train = X_train[train_filter], y_train[train_filter]
X_test, y_test = X_test[test_filter], y_test[test_filter]"""

y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

X_train, X_test = X_train / 255.0, X_test / 255.0
X_train, X_test = (
    X_train.reshape(X_train.shape[0], 1, X_train.shape[1], X_train.shape[2]),
    X_test.reshape(X_test.shape[0], 1, X_test.shape[1], X_test.shape[2]),
)

print(X_train.shape, X_test.shape)
print((y_train.shape, y_test.shape))

(60000, 1, 28, 28) (10000, 1, 28, 28)
((60000, 10), (10000, 10))


In [3]:
class MNISTER(Module):
    def __init__(self):
        super().__init__()
        self.conv2d_1 = Conv2D(X_train.shape[1:], 9, 5)
        self.sigmoid_1 = Sigmoid()
        self.reshape_1 = Reshape((1, 2000))
        self.dense_1 = Dense(2000, 784)
        self.sigmoid_2 = Sigmoid()
        self.dense_2 = Dense(784, 256)
        self.sigmoid_3 = Sigmoid()
        self.dense_3 = Dense(256, 10)
        self.softmax = Softmax()

    def forward(self, x_sample: Tensor) -> Tensor:
        out = self.conv2d_1(x_sample)
        out = self.sigmoid_1(out)
        out = self.reshape_1(out)
        out = self.dense_1(out)
        out = self.sigmoid_2(out)
        out = self.dense_2(out)
        out = self.sigmoid_3(out)
        out = self.dense_3(out)
        out = self.softmax(out)
        return out

    def __call__(self, x_sample: Tensor) -> Tensor:
        return self.forward(x_sample)
    
    def save(self, file_path: str) -> None:
        with open(file_path, "wb") as fh:
            pickle.dump(self, fh)

In [4]:
model = MNISTER()
optim = SGD(model.params, lr=0.01, regulization=0)
loss_func = CrossEntropy()

epochs = 3

In [5]:
for x in range(0, 5):
    pred = model(X_test[x])
    print(f"Pred: {np.argmax(pred.data)}. True: {np.argmax(y_test[x])}")

trues = np.argmax(y_test, axis=-1)

preds = []
for x in range(0, X_test.shape[0]):
    with no_grad():
        pred = model(X_test[x])
        preds.append(np.argmax(pred.data))

accuracy = np.array((preds == trues)).astype(int)

pct_score = accuracy.mean() * 100

print(f"Dataset Percentage Accuracy: {pct_score}")

Pred: 0. True: 7
Pred: 7. True: 2
Pred: 7. True: 1
Pred: 7. True: 0
Pred: 0. True: 4
Dataset Percentage Accuracy: 8.33


In [6]:
model.save("mnister.pkl")

In [7]:
loaded_model = MNISTER.load("mnister.pkl")

for x in range(0, 5):
    pred = loaded_model(X_test[x])
    print(f"Pred: {np.argmax(pred.data)}. True: {np.argmax(y_test[x])}")

trues = np.argmax(y_test, axis=-1)

Pred: 0. True: 7
Pred: 7. True: 2
Pred: 7. True: 1
Pred: 7. True: 0
Pred: 0. True: 4


In [ ]:
for epoch in range(epochs):
    print(f"---------EPOCH: {epoch + 1}------------")

    loss = Tensor(0, requires_grad=True)
    for sample in tqdm(range(0, X_train.shape[0])):
        pred = model(X_train[sample])
        loss = loss_func(pred.reshape((10, 1)), Tensor(y_train[sample].reshape(10, 1)))
        loss.backward()
        optim.step()
        optim.zero_grad()

    gc.collect()

    with no_grad():
        
        loss = 0
        preds = []
        trues = np.argmax(y_test, axis=-1)

        for sample in tqdm(range(0, X_test.shape[0])):
            pred = model(X_test[sample])
            preds.append(np.argmax(pred.data))
            loss += loss_func(
                pred.reshape((10, 1)), Tensor(y_test[sample].reshape(10, 1))
            )
    
        accuracy = np.array((preds == trues)).astype(int)

        pct_score = accuracy.mean() * 100

        loss /= X_test.shape[0]

        print(f"TEST LOSS: {loss} - Accuracy: {pct_score}")

    gc.collect()

---------EPOCH: 1------------


100%|██████████| 10000/10000 [01:23<00:00, 119.53it/s]


TEST LOSS: Tensor([0.50855749], shape = (1,)) - Accuracy: 84.48
---------EPOCH: 2------------


100%|██████████| 10000/10000 [00:49<00:00, 201.49it/s]


TEST LOSS: Tensor([0.42650206], shape = (1,)) - Accuracy: 87.87
---------EPOCH: 3------------


100%|██████████| 10000/10000 [00:54<00:00, 183.15it/s]


TEST LOSS: Tensor([0.41278366], shape = (1,)) - Accuracy: 87.99
